In [ ]:
import os
import numpy as np

In [ ]:
from cmdstanpy import cmdstan_path, set_cmdstan_path

# set the path to the CmdStan installation
system_drive = os.environ.get("SystemDrive", "C:")
print(system_drive)
path_to_cmdstan = os.path.join(system_drive + os.sep, "Users", "fonta", "cmdstan")
print(path_to_cmdstan)
set_cmdstan_path(path_to_cmdstan)

cmdstan_path()

In [ ]:
from cmdstanpy import CmdStanModel


In [ ]:
# for reproducibility
__NP_SEED__ = 23456789
np.random.seed(__NP_SEED__)

# Code-along Chapter 16

## Code 1

First example from pp. 371

In [ ]:
MODEL_filename = "code1_pp371.stan"

In [ ]:
MCMC_chains = 4
MCMC_warm_up_iters = 100
MCMC_sampling_iters = 200

In [ ]:
true_mu = 1.5
true_sigma = 0.2
num_samples = 100
y_data = np.random.normal(loc=true_mu, scale=true_sigma, size=num_samples)
data = {'N': num_samples, 'Y': y_data}

In [ ]:
# Instantiate a STAN model
stan_model_parent_dir = os.getcwd()
print(stan_model_parent_dir)
model = CmdStanModel(stan_file=os.path.join(stan_model_parent_dir, MODEL_filename))
print(model)
print(model.exe_info())

In [ ]:
data

In [ ]:
fit = model.sample(
    data=data,
    chains=MCMC_chains,
    iter_warmup=MCMC_warm_up_iters,
    iter_sampling=MCMC_sampling_iters,
    show_console=True
)
fit

In [ ]:
fit.summary()